In [4]:
from keras.preprocessing import image
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os
from zipfile import ZipFile
from google.colab import drive
import tempfile

tmpdir = tempfile.mkdtemp()

In [9]:
# Commented out lines import tsb.zip into google colab environment
# Need to upload the file to your google drive, below file path is in the root directory

# mount drive
drive.mount('/content/gdrive')

# used for unzip files
!unzip gdrive/My\ Drive/ava.zip


# path to a image: tsb/basedata/training/bad-image/2.jpg
img = tf.keras.preprocessing.image.load_img('/content/ava/Training/training_bad_image/2.jpg')
# save the image to the current directory
img.save('2.jpg')

cv2.imread('2.jpg')
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Archive:  gdrive/My Drive/ava.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of gdrive/My Drive/ava.zip or
        gdrive/My Drive/ava.zip.zip, and cannot find gdrive/My Drive/ava.zip.ZIP, period.


FileNotFoundError: ignored

In [ ]:
training_dataset = train.flow_from_directory('/content/ava/basedata/training',
                                            target_size=(200,200),
                                            batch_size=64,
                                            class_mode='binary')

validation_dataset = train.flow_from_directory('/content/ava/basedata/Validating',
                                            target_size=(200,200),
                                            batch_size=64,
                                            class_mode='binary')

In [ ]:
training_dataset.class_indices


In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(200,200,3)),
                                      tf.keras.layers.MaxPooling2D(2,2),
                                      # The second convolution
                                        tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                                        tf.keras.layers.MaxPooling2D(2,2),
                                        # The third convolution
                                        tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                        tf.keras.layers.MaxPooling2D(2,2),
                                        # The fourth convolution
                                        tf.keras.layers.Flatten(), 
                                        tf.keras.layers.Dense(512,activation='relu'),
                                        tf.keras.layers.Dense(1,activation='sigmoid')])
model.summary()


In [ ]:
model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(lr=0.001),
                metrics=['accuracy'])

In [ ]:
model.fit(training_dataset,
            steps_per_epoch=19,
            epochs=10,
            validation_data=validation_dataset)

In [ ]:
dir_path = '/content/ava/basedata/testing'
for img in os.listdir(dir_path):
    img = tf.keras.preprocessing.image.load_img(dir_path + '/' + img, target_size=(200,200))
    plt.imshow(img)
    plt.show()
    X = tf.keras.preprocessing.image.img_to_array(img)
    X = np.expand_dims(X, axis=0)

    images = np.vstack([X])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    if classes[0]==0:
        print('bad photo')
    else:
        print('good photo')

In [ ]:
# predict the image in the custom-ava-downloader/validation_image folder with the model
# print the prediction result
# if the prediction result is 1, then the image is a good image
# if the prediction result is 0, then the image is a bad image
import numpy as np
import json
from keras.preprocessing import image

# read JSON file, for comparsion later on with prediction result
with open('/ava/image_data.json') as json_file:
    data = json.load(json_file)

# loop through the images in the custom-ava-downloader/validation_image folder and predict the image
for image_name in os.listdir('ava/Testing'):

    # resize this image 
    test_image = image.load_img('ava/Testing/' + image_name, target_size = (200, 200))

    # print the image 
    plt.imshow(test_image)
    plt.show()

    # convert the image to an array
    test_image = image.img_to_array(test_image)

    # expand the dimension of the image
    test_image = np.expand_dims(test_image, axis = 0)

    # predict the image
    result = model.predict(test_image)
    if result[0][0] == 1:
        prediction = 'good image'
    else:
        prediction = 'bad image'
    print(image_name + " is a " + prediction + " with a prediction result of " + str(result[0][0]))

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)